## <b><i>Desafio complementario: Evaluando modelos</i></b>
Autor: <i>Emanuel Benitez</i><br>
Fecha: <i>03-11-2023</i>

#### <b>Consigna</b>

<ol>
    <li> Generar una evaluación de modelos.</li>
    <li> Identificar por medio de las métricas generadas si se puede tener una situación de overfitting o underfitting (subajuste), discutiendo posibles formas de mejora. </li>
</ol>

In [29]:
# Comenzamos importando las librerias necesarias para el análisis de datos

# para manipulación de datos y calculos
import pandas as pd
import numpy as np

# para la selección de caracteristicas
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# para el modelo de regresión k-NN:
from sklearn.neighbors import KNeighborsRegressor

# para la validación cruzada de k-NN:
from sklearn.model_selection import cross_val_score


# para el modelo de regresión lineal:
from sklearn.linear_model import LinearRegression

# para separar en entrenamiento y prueba: 
from sklearn.model_selection import train_test_split

# para evaluar el resultado de los modelos
from sklearn.metrics import mean_squared_error, r2_score


url = "https://raw.githubusercontent.com/emanuelbe1/ciencia_de_datos_repasos/main/Coderhouse/Datasets/datos/procesados/Cuantitative_spotify_data.csv"

df_spotify = pd.read_csv(url)
df_spotify.drop(columns = 'Unnamed: 0', inplace = True)

<p>&nbsp;&nbsp;&nbsp;&nbsp;Estos datos son parte del mismo conjunto de datos usados para el trabajo principal ("Proyecto_ParteIII"). Aisle esta sección de interés para claridad, y para evaluar dos modelos distintos: k-NN y Regresión Lineal.</p>
<p>&nbsp;En estos datos se elimino algunos valores inconsistentes y contiene las columnas cuantitativas de las canciones en Spotify. No contiene otras variables cómo el número de vistas en Youtube, descripción de la canción o número de comentarios.</p>
<p>El punto de interes es ver si hay alguna mejora relativa en la predicción o las capacidades del modelo, considerando las metricas de uno u otro.</p>

In [30]:
df_spotify.head()

,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Valence,Tempo,Duration_ms,Stream
0,0.818,0.705,-6.679,0.1770,0.008360,0.002330,0.772,138.559,222640.0,1.040235e+09
1,0.676,0.703,-5.815,0.0302,0.086900,0.000687,0.852,92.761,200173.0,3.100837e+08
2,0.695,0.923,-3.930,0.0522,0.042500,0.046900,0.551,108.014,215150.0,6.306347e+07
3,0.689,0.739,-5.810,0.0260,0.000015,0.509000,0.578,120.423,233867.0,4.346636e+08
4,0.663,0.694,-8.627,0.1710,0.025300,0.000000,0.525,167.953,340920.0,6.172597e+08


<b>¿Por qué comparar la regresión k-NN con la regresión lineal?</b><br>
 <p>En el trabajo anterior, observe que no hay una relación lineal en los datos. En motivo de responder a la consigna, decidi demostrarlo esta falta de relación modelando una recta que intente ajustarse, para recibir un coeficiente de determinación casi inexistente: <b>0.03</b>. Esto era esperable debido a la naturaleza no lineal de la relación.</p>
 <p> El modelo k-NN se basa en que tan similares son los puntos entre datos cercanos en el espacio de las características. <b>No asume una relación lineal</b> entre las caracteristicas y la variable objetivo ('Streams'), lo que puede hacerlo más útil cuando la relación es no-lineal.</p>
 <p> El modelo de regresión lineal asume este tipo de relación entre las caracteristicas y la variable objetivo. Si la relación fuera lineal, seria más indicado este tipo de modelo.</p>
 
 
 <p>&nbsp;Por este motivo, mi hipotesis es que el modelo de regresión k-NN muestre un coeficiente de determinación más alto que la regresión lineal, o un MSE más bajo. Sin embargo, quiero <b>cuantificar</b> esa mejora.</p>


## <center>Generar una evaluación de modelos</center>

<center><b> Regresión lineal </b></center>
<p>&nbsp;Primero voy a modelar una regresión lineal para buscar cúal es el mejor número de caracteristicas. Para eso voy a iterar Y cada vez seleccionar las mejores caracteristicas para el número dado.</p>
Las caracteristicas que quiero probar son:

- Danceability
- Energy
- Speechiness
- Acousticness
- Instrumentalness
- Valence

Estás son siete caracteristicas. En el siguiente código calculo el MSE y R² para cada número de variables seleccionadas (de 1 a 6), y respondo a la pregunta: ¿cúal es el mejor número de caracteristicas?.

1. Selecciono las caracteristicas <b>X</b> y la variable objetivo <b>y</b>

In [31]:
# Las variables Loudness, Tempo, y Duration_ms las quite. Estas no representan una relación lineal

X = df_spotify.drop(columns = ['Loudness', 'Tempo', 'Duration_ms', 'Stream'])
y = df_spotify[['Stream']]

X.head()

,Danceability,Energy,Speechiness,Acousticness,Instrumentalness,Valence
0,0.818,0.705,0.1770,0.008360,0.002330,0.772
1,0.676,0.703,0.0302,0.086900,0.000687,0.852
2,0.695,0.923,0.0522,0.042500,0.046900,0.551
3,0.689,0.739,0.0260,0.000015,0.509000,0.578
4,0.663,0.694,0.1710,0.025300,0.000000,0.525


2. Divido en conjuntos para el entrenamiento y el testeo con una proporción del 80% para el entrenamiento

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 38)

3. Ahora voy a iterar seis veces sobre usando el metodo SFS con k_features = n, donde n sera el número de caracteristicas, y ver los resultados de MSE.

In [33]:
from sklearn.model_selection import cross_val_score

In [34]:
modelo_lineal = LinearRegression()
for n in range(1, 7):
    sfs = SFS(modelo_lineal, k_features=n, scoring='r2')
    sfs.fit(X_train, y_train)
    
    caracteristicas_elegidas = list(sfs.k_feature_names_)
    
    modelo_lineal.fit(X_train[caracteristicas_elegidas], y_train)

    y_predichos = modelo_lineal.predict(X_test[caracteristicas_elegidas])
    
        
    mse = mean_squared_error(y_test, y_predichos)
    r2 = r2_score(y_test, y_predichos)
    
    print(f"Modelo lineal con {n} variables")
    print(f"Caracteristicas más descriptivas: {caracteristicas_elegidas}")
    print(f"Valor R² del modelo: {r2} ")
    print(f"Valor MSE del modelo: {mse} ")
    print("____"*15, "\n")

Modelo lineal con 1 variables
Caracteristicas más descriptivas: ['Acousticness']
Valor R² del modelo: 0.0073786037252512315 
Valor MSE del modelo: 6.511305373025306e+16 
____________________________________________________________ 

Modelo lineal con 2 variables
Caracteristicas más descriptivas: ['Acousticness', 'Instrumentalness']
Valor R² del modelo: 0.011496205795018088 
Valor MSE del modelo: 6.4842951105209176e+16 
____________________________________________________________ 

Modelo lineal con 3 variables
Caracteristicas más descriptivas: ['Acousticness', 'Instrumentalness', 'Valence']
Valor R² del modelo: 0.013128041271966429 
Valor MSE del modelo: 6.473590748164008e+16 
____________________________________________________________ 

Modelo lineal con 4 variables
Caracteristicas más descriptivas: ['Danceability', 'Acousticness', 'Instrumentalness', 'Valence']
Valor R² del modelo: 0.017113295677146922 
Valor MSE del modelo: 6.4474486475416456e+16 
__________________________________

4. Podemos ver que el modelo lineal esta muy lejos de ser el indicado para predecir el número de streams, y que el r² es casi cercano a cero. Si bien muestra cierta mejora al alcanzar las 4 variables: Dancebility, Acousticness, Instrumentalness, y Valence, al continuar agregando caracteristicas, el r² se mantiene casi igual, mientras que el MSE continua en una magnitud extremadamente alta. Este MSE indica que hay una relación no lineal entre los datos y que el modelo de regresión lineal no es el adecuado. 

<br>
<center><b>Modelo de regresión k-NN</b></center>

<p>&nbsp;Vemos que el modelo de regresión lineal no es efectivo para explicar las tendencias en el acto. Ahora, considerando que el modelo knn es no-parametrico y no asume una relación lineal en los datos, podría ver que tan efectivo es para predecir el número de reproducciones, y si es más indicado para estos datos.</p>

<p>&nbsp;En este tipo de modelo, hay dos números que debo considerar: </p>

- El número de caracteristicas del modelo
    
- El número optimo <b>k</b> de vecinos


<p> En este caso voy a tomar un enfoque similar al anterior: voy a tomar las seis variables:</p>
    
- Danceability
- Energy
- Speechiness
- Acousticness
- Instrumentalness
- Valence

Y ver el número de caracteristicas optimo, y ver el resultado después de aplicar validación cruzada a cada número seleccionado. Después de hacer la selección de caracteristicas, voy a probar distintos valores de <b>k</b> en el modelo knn, que reduzca el MSE para los datos de testeo.

In [35]:
# Las caracteristicas ya fueron seleccionadas en la sección anterior: Regresión lineal
# de la misma manera, también se encuentra el número de Stream (variable objetivo) en la variable y
# Estos datos también fueron separados en X_train, y_train, X_test e y_test.
X_train.head()

,Danceability,Energy,Speechiness,Acousticness,Instrumentalness,Valence
17662,0.894,0.528,0.4660,0.04440,0.000000,0.200
13366,0.686,0.862,0.0340,0.00207,0.000011,0.472
16559,0.905,0.550,0.0674,0.23900,0.119000,0.362
3501,0.524,0.345,0.0261,0.48900,0.013000,0.771
8296,0.692,0.799,0.1640,0.38800,0.000000,0.549


1. Primero voy a buscar el número de carácteristicas con el que voy a entrenar el modelo. Para aplicar la selección secuencial de caracteristicas (SFS), voy a empezar con un valor inicial de k = 5 para el modelo k-NN.

In [36]:
# Instancio el modelo knn
modelo_knn = KNeighborsRegressor(n_neighbors= 5)


for n in range(1, 7):
    sfs = SFS(modelo_knn, k_features=n, scoring='neg_mean_squared_error')
    sfs.fit(X_train, y_train)
    
    caracteristicas_elegidas = list(sfs.k_feature_names_)    
    
    puntaje_mse = sfs.k_score_
    
    print(f"Modelo k-NN con {n} variables")
    print(f"Caracteristicas más descriptivas: {caracteristicas_elegidas}")
    print(f"Valor MSE del modelo: {-puntaje_mse} ")
    print("____"*15, "\n")

Modelo k-NN con 1 variables
Caracteristicas más descriptivas: ['Instrumentalness']
Valor MSE del modelo: 6.364515518711369e+16 
____________________________________________________________ 

Modelo k-NN con 2 variables
Caracteristicas más descriptivas: ['Acousticness', 'Instrumentalness']
Valor MSE del modelo: 6.641636093432821e+16 
____________________________________________________________ 

Modelo k-NN con 3 variables
Caracteristicas más descriptivas: ['Speechiness', 'Acousticness', 'Instrumentalness']
Valor MSE del modelo: 6.51771528790502e+16 
____________________________________________________________ 

Modelo k-NN con 4 variables
Caracteristicas más descriptivas: ['Energy', 'Speechiness', 'Acousticness', 'Instrumentalness']
Valor MSE del modelo: 6.5521327539017704e+16 
____________________________________________________________ 

Modelo k-NN con 5 variables
Caracteristicas más descriptivas: ['Danceability', 'Energy', 'Speechiness', 'Acousticness', 'Instrumentalness']
Valor MS

<p>Se puede ver que el modelo de regresión k-NN con el menor MSE para k = 5  es el entrenado con la caracteristica "Instrumentalness". Esto sigue conteniendo un ECM muy alto, pero parece indicar que una buena caracteristica a considerar es esta en cuanto a popularidad de una canción.</p>

2. Ahora, el siguiente paso es hacer un bucle for para ver el optimo valor de k.


In [37]:
for n in range(1, 30):
    modelo_knn = KNeighborsRegressor(n_neighbors= n)  
    scores = cross_val_score(modelo_knn, X_train[['Instrumentalness']], y_train, cv=10, scoring='neg_mean_squared_error')
    
    print(f'Para k = {n}, el valor MSE es: {scores.mean()}')
    print('____'*15)

Para k = 1, el valor MSE es: -9.51084427777986e+16
____________________________________________________________
Para k = 2, el valor MSE es: -7.303134815695806e+16
____________________________________________________________
Para k = 3, el valor MSE es: -6.877960036237554e+16
____________________________________________________________
Para k = 4, el valor MSE es: -6.553404367633796e+16
____________________________________________________________
Para k = 5, el valor MSE es: -6.416658125242193e+16
____________________________________________________________
Para k = 6, el valor MSE es: -6.321461819747707e+16
____________________________________________________________
Para k = 7, el valor MSE es: -6.25708755338717e+16
____________________________________________________________
Para k = 8, el valor MSE es: -6.157737640389133e+16
____________________________________________________________
Para k = 9, el valor MSE es: -6.158951285243413e+16
______________________________________________

Se puede ver que un valor optimo para k es <b>16</b>, donde el error del modelo empieza a mostrar cierta estabilidad, a medida que se aumenta el valor de k.

3. Ahora entreno el modelo final con k = 16, usando la caracteristica elegida "Instrumentalness", y calculo metricas en el sub-conjunto de testeo.

In [41]:
# Instancio el modelo con k = 16:
modelo_knn = KNeighborsRegressor(n_neighbors= 16)  


# Entreno con la caracteristica elegida:
modelo_knn.fit(X_train[['Instrumentalness']], y_train)

# Genero predicciones para el conjunto de testeo
y_predichos = modelo_knn.predict(X_test[['Instrumentalness']])


# Calculo el MSE del modelo en relación a los datos de testeo
mse = mean_squared_error(y_test, y_predichos)

# Calculo el coeficiente de determinación en relación a los datos de testeo
r2 = r2_score(y_test, y_predichos)

print(f"Error Cuadrático Medio (MSE): {mse}")
print(f"Coeficiente de Determinación (R²): {r2}")

Error Cuadrático Medio (MSE): 6.850304019379547e+16
Coeficiente de Determinación (R²): -0.044300328593515514


### Identificar por medio de las métricas generadas si se puede tener una situación de overfitting o underfitting (subajuste), discutiendo posibles formas de mejora.

<p>&nbsp;&nbsp;&nbsp;&nbsp;En los ejemplos anteriores de Regresión lineal y Regresión k-NN se puede observar que los patrones observados en los datos no se capturan con este tipo de modelos. El alto error cuadratico medio y el bajo r² demuestran que estos no son los más indicados para análizar una posible relación. La situación de underfitting es esperable: los modelos no capturan el comportamiento de los datos, y no predicen el resultado de "streams" con las variables presentes. Quizás los patrones presentes en estos datos puedan ser análizados por medio de aplicar otro tipo de modelos (quizás agrupamiento k-Means).</p>
<p>&nbsp; Se puede también considerar un enfoque alternativo: predecir el número de comentarios a partir de la cantidad de visualizaciones en Youtube, que tenga una canción. En este otro enfoque, se puede aplicar una regresión logística para la clasificación entre "Muchos  comentarios" y "Pocos comentarios" (para hacer un análisis relativo). Entonces, la variable X predictora es el número de "views", y la variable y seria 0 (Pocos comentarios y 1 (Muchos comentarios), para el calculo de la probabilidad en base a las visualizaciones.</p>
<br>
<p>&nbsp; En resumen, los modelos propuestos no son los indicados para demostrar que una o más de las caracteristicas disponibles en este conjunto de datos explique cierto número de reproducciones.</p>